In [1]:
# Iz final datotek naredi file v katerem so shranjeni vsi podatki

import pandas as pd

save_directory = "C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Other\\"

df = pd.read_excel(save_directory + "final_data_avtonet1.xlsx" )
for i in range(2,11):
    df2 = pd.read_excel(save_directory + "final_data_avtonet" + str(i) + ".xlsx" )
    df = df.append(df2)
df = df.drop(columns=['Unnamed: 0'])

df.to_excel("C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Final data\\Data.xlsx")




In [4]:
# Iz final datotek naredi file v katerem so shranjeni vsi podatki
import pandas as pd
import numpy as np
import random
import statistics
import time
data = pd.read_excel("C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Final data\\Data.xlsx")


In [3]:
# Pri nekaterih avtih manjka leto proizvodnje pri nekaterih leto prve registracije
# Podatek starost shrani v novi stolpec

leto_proizvodnje = data["leto_proizvodnje"]
leto_prve_registracije = data["leto_prve_registracije"]
starost = []
for i in range(0,len(leto_proizvodnje)):

    if (str(leto_proizvodnje[i]) == "nan") and (str(leto_prve_registracije[i]) == "nan"):
        starost.append(leto_proizvodnje[i])
        
    elif str(leto_proizvodnje[i]) == "nan":
        starost.append(2022 - int(leto_prve_registracije[i]))
        
    elif str(leto_prve_registracije[i]) == "nan":
        starost.append(2022 - int(leto_proizvodnje[i]))        
    else:
        st = max(int(leto_proizvodnje[i]),int(leto_prve_registracije[i]))
        starost.append(2022-st)
        
data["starost"] = starost


# Dodaj ID- je 
data["ID"] = [i for i in range(1,len(starost) + 1)]

# Razdeli na train in test set, za test vzami samo podatke, ki nimajo mankajočih vrednosti.

data_test = data.dropna().sample(1000)

data_sample_ids = list(data_test["ID"].values)

data_train = data[~data.ID.isin(data_sample_ids)]

Train_test_identificator = []

for i in range(0,len(leto_proizvodnje)):
    if (i+1) in data_sample_ids:
        Train_test_identificator.append("test")
    else:
        Train_test_identificator.append("train")
    
data["Train_test_identificator"] = Train_test_identificator

# V modelu bomo moč motorjo pokategorizirali glede na decile, za razdelitev moramo uporabiti podatke iz train seta

# Ker želimo, povprečja decile in podobnie stvari računati le iz train seta, na tem delu testa ne upoštevamo.
data_train = data[data.Train_test_identificator == "train"]
data_test = data[data.Train_test_identificator == "test"]

decili_ccm = np.array([data_train["ccm"].quantile( i / 10) for i in range(0,11)])
decili_kw = np.array([data_train["kw"].quantile( i / 10) for i in range(0,11)])

# Shrani decile v dataframe
decili_ccm_kw = pd.DataFrame({"decili_ccm" : decili_ccm, "decili_kw" : decili_kw})


ccm_classes =[]
kw_classes = []

for index, row in data_train.iterrows(): 
    ccm = row["ccm"]
    kw = row["kw"]
    
    ccm_classes.append(sum(np.array(decili_ccm) < ccm))
    kw_classes.append(sum(np.array(decili_kw) < kw))
    
data_train["ccm_class"] =ccm_classes
data_train["kw_class"] = kw_classes

ccm_classes =[]
kw_classes = []

for index, row in data_test.iterrows(): 
    ccm = row["ccm"]
    kw = row["kw"]
    
    ccm_classes.append(sum(np.array(decili_ccm) < ccm))
    kw_classes.append(sum(np.array(decili_kw) < kw))
    
data_test["ccm_class"] =ccm_classes
data_test["kw_class"] = kw_classes


data = data_test.append(data_train)
data = data.reset_index()


# Izračunaj statistike glede na tipe avtomobila

mean_by_type = data_train.groupby(['tip', "znamka"])["cena"].mean().reset_index(name ="mean")
count_by_type = data_train.groupby(['tip', "znamka"]).size().reset_index(name='count')
sd_by_type = data_train.groupby(['tip', "znamka"])["cena"].std().reset_index(name='std')


group_data = (mean_by_type.merge(count_by_type, on =["tip", "znamka"])).merge(sd_by_type, on =["tip", "znamka"])

# Odstrani avte s countom manjšim od 3

group_data = group_data[group_data["count"] >=  3]


# Shrani izračunano

group_data.to_excel("C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Final data\\Group_Data.xlsx")
decili_ccm_kw.to_excel("C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Final data\\decili_ccm_kw.xlsx")
data.to_excel("C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Final data\\Final_data.xlsx")



ValueError: a must be greater than 0 unless no samples are taken

In [39]:
name_of_file = "C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Final data\\Data_statistics.xlsx"

with pd.ExcelWriter(name_of_file) as writer: 

    data.to_excel(writer, sheet_name = "Data")

    data.describe(percentiles = [0.05,0.25,0.5,0.75,0.95])

    data.describe(percentiles = [0.05,0.25,0.5,0.75,0.95]).to_excel(writer, sheet_name = "Descriptive Statistics")

    data['stevilo_vrat'].value_counts()

    data['stevilo_vrat'].value_counts().to_excel(writer, sheet_name = "Stevilo vrat Frequency")

    data['znamka'].value_counts()

    data['znamka'].value_counts().to_excel(writer, sheet_name = "Znamka Frequency")

    data['tip'].value_counts()

    data['tip'].value_counts().to_excel(writer, sheet_name = "Tip Frequency")

    data['gorivo'].value_counts()

    data['gorivo'].value_counts().to_excel(writer, sheet_name = "Gorivo Frequency")

    data['menjalnik'].value_counts()

    data['menjalnik'].value_counts().to_excel(writer, sheet_name = "Menjalnik Frequency")

    data['oblika'].value_counts()

    data['oblika'].value_counts().to_excel(writer, sheet_name = "Oblika Frequency")

In [37]:
# Dodaj še razvrstitev tipov po povprečni ceni

data2 = data[data["Train_test_identificator"] == "train"]

means = data2.groupby("tip").mean()
cene = list(means["cena"].values)
tipi = list(means.index)

df = pd.DataFrame({"tip" : tipi, "cena" : cene}).sort_values(by="cena")

df["razvrstitev_povprecne_cene_po_tipu"] = [i for i in range(1,(len(df.index)+1))]

df


data_out = data.merge(df, on = "tip", how = "left")
data_out = data_out.rename(columns = {"cena_x" :"cena"})
name_of_file = "C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Final data\\Final_data.xlsx"
data_out.to_excel(name_of_file)


In [36]:
data_out.rename(columns = {"cena_x" :"cena"})

,cena,znamka,tip,leto_prve_registracije,leto_proizvodnje,prevozeni_km,leto_veljavnosti_tehnicnega_pregleda,stevilo_vrat,kw,ccm,...,oblika,barva,emisije_c02,starost,ID,Train_test_identificator,ccm_class,kw_class,cena_y,razvrstitev_povprecne_cene_po_tipu
0,3790,Lancia,Delta,2011.0,2011.0,314000.0,2022.0,5,121.0,1956.0,...,kombilimuzina,\r\n siva metalik\r\n,139.0,11.0,9,test,6,8,4214.500000,109.0
1,19990,Audi,A6 Avant,2015.0,2015.0,140000.0,2022.0,5,140.0,1968.0,...,karavan,\r\n bela\r\n,114.0,7.0,11,test,6,9,19255.050000,348.0
2,24990,Mercedes-Benz,C-Razred,2015.0,2015.0,118050.0,2023.0,4,125.0,2143.0,...,limuzina,\r\n èrna metalik\r\n,103.0,7.0,14,test,9,8,16253.145299,327.0
3,6990,Renault,Twingo,2016.0,2016.0,70000.0,2022.0,5,52.0,999.0,...,kombilimuzina,\r\n rdeèa metalik\r\n,112.0,6.0,19,test,1,1,4304.820896,110.0
4,3399,Volkswagen,Golf,2007.0,2007.0,238000.0,2022.0,5,66.0,1896.0,...,kombilimuzina,\n,135.0,15.0,24,test,6,2,11661.851852,265.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10207,24990,Skoda,Superb,NaN,2017.0,104000.0,2024.0,5,140.0,1968.0,...,karavan,\r\n rjava metalik\r\n,136.0,5.0,10208,train,6,9,18447.438596,342.0
10208,13800,Volkswagen,Golf Variant,2013.0,2013.0,110000.0,2022.0,5,110.0,1968.0,...,karavan,\n,124.0,9.0,10209,train,6,7,11014.898148,255.0
10209,500,Peugeot,307,2004.0,2004.0,295000.0,2022.0,5,66.0,1997.0,...,karavan,\n,138.0,18.0,10210,train,9,2,1488.611111,40.0
10210,10999,Audi,Q7,NaN,2008.0,382000.0,2022.0,5,176.0,2967.0,...,terensko vozilo,\r\n bela\r\n,239.0,14.0,10211,train,10,10,31644.472222,435.0
